In [1]:
# !pip install gensim --user

In [366]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

In [505]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [3]:
import re
import numpy as np
import pandas as pd
from string import punctuation
from PreProcessing.Opentext import get_EBS

['lec00001_EBS.txt', 'lec00002_EBS.txt', 'lec00003_EBS.txt', 'lec00004_EBS.txt', 'lec00005_EBS.txt', 'lec00006_EBS.txt', 'lec00007_EBS.txt', 'lec00008_EBS.txt', 'lec00009_EBS.txt', 'lec00010_EBS.txt', 'lec00011_EBS.txt', 'lec00012_EBS.txt', 'lec00013_EBS.txt', 'lec00014_EBS.txt', 'lec00015_EBS.txt', 'lec00016_EBS.txt', 'lec00017_EBS.txt', 'lec00018_EBS.txt', 'lec00019_EBS.txt', 'lec00020_EBS.txt', 'lec00021_EBS.txt', 'lec00022_EBS.txt', 'lec00023_EBS.txt', 'lec00024_EBS.txt', 'lec00025_EBS.txt', 'lec00026_EBS.txt']


In [4]:
from jamo import h2j, j2hcj, j2h
from soynlp.hangle import levenshtein, jamo_levenshtein
import nltk
from nltk import Text
from nltk.tag import pos_tag
from nltk.corpus import gutenberg
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer, regexp_tokenize
from nltk.help import upenn_tagset
from konlpy.corpus import kolaw, kobill
from konlpy.tag import Kkma, Mecab, Okt, Komoran, Hannanum
from PreProcessing.find_common_part import find_common_part
from PreProcessing.Dictionary import sd, ad, kopro
from PreProcessing.Opentext import get_EBS, get_EBS_entered, get_STT1_Google_entered, get_STT1_Google, get_STT1_Transcribe_entered
from collections import defaultdict
pd.set_option("display.max_rows", None, "display.max_columns", None)
from konlpy.tag import Kkma, Mecab, Okt, Komoran, Hannanum

0
0
0
1
2
5
4
8


In [5]:
EBS = get_EBS(1, 26)

['lec00001_EBS.txt', 'lec00002_EBS.txt', 'lec00003_EBS.txt', 'lec00004_EBS.txt', 'lec00005_EBS.txt', 'lec00006_EBS.txt', 'lec00007_EBS.txt', 'lec00008_EBS.txt', 'lec00009_EBS.txt', 'lec00010_EBS.txt', 'lec00011_EBS.txt', 'lec00012_EBS.txt', 'lec00013_EBS.txt', 'lec00014_EBS.txt', 'lec00015_EBS.txt', 'lec00016_EBS.txt', 'lec00017_EBS.txt', 'lec00018_EBS.txt', 'lec00019_EBS.txt', 'lec00020_EBS.txt', 'lec00021_EBS.txt', 'lec00022_EBS.txt', 'lec00023_EBS.txt', 'lec00024_EBS.txt', 'lec00025_EBS.txt', 'lec00026_EBS.txt']


In [6]:
GOOGLE = get_STT1_Google(1, 26)

['lec00001_STT1_Google.txt', 'lec00002_STT1_Google.txt', 'lec00003_STT1_Google.txt', 'lec00004_STT1_Google.txt', 'lec00005_STT1_Google.txt', 'lec00006_STT1_Google.txt', 'lec00007_STT1_Google.txt', 'lec00008_STT1_Google.txt', 'lec00009_STT1_Google.txt', 'lec00010_STT1_Google.txt', 'lec00011_STT1_Google.txt', 'lec00012_STT1_Google.txt', 'lec00013_STT1_Google.txt', 'lec00014_STT1_Google.txt', 'lec00015_STT1_Google.txt', 'lec00016_STT1_Google.txt', 'lec00017_STT1_Google.txt', 'lec00018_STT1_Google.txt', 'lec00019_STT1_Google.txt', 'lec00020_STT1_Google.txt', 'lec00021_STT1_Google.txt', 'lec00022_STT1_Google.txt', 'lec00023_STT1_Google.txt', 'lec00024_STT1_Google.txt', 'lec00025_STT1_Google.txt', 'lec00026_STT1_Google.txt']


In [7]:
EBS[1070:3000]

'게 수선을 똑같이 그어 주면 바로 두 직각삼각형 ABC하고, ADE는 서로 어떤 닮음?AA닮음이 된다. 이건 우리가 중학교 2학년 때 배웠죠, 그렇죠? ∠A가 공통이고, 그다음에 둘 다 직각을 갖고 있으니까 AA닮음이 될 테고.우리가 중2 때 배웠던 닮음비라는 건 뭐였냐 하면 대응하는 변의 길이의 비였어요, 그렇죠? 그래서 만약에 이 길이를 5라고 해 보고, 이 길이를 10이라고 해 보면.두 삼각형의 닮음비는 5:10. 1:2겠죠. 그렇다면 우리가 이 길이를 4라고 놔 보면, 이 길이는 얼마가 됩니까, 당연히? 그렇죠. 4의 두 배인 8이 돼야 돼.이 길이를 거꾸로 우리가 6이라고 해 보겠습니다. 그럼 거꾸로 이 작은 쪽은 얼마가 될까? 1:2, 2:1. 얼마? 3이 돼야 된다는 걸 여러분들이 알 수 있죠.그래서 우리는 닮음비라는 건 ‘작은 삼각형과 큰 삼각형의 대응하는 변의 길이의 비’이니까 5:10, 3:6, 4:8. 이것이 모두 다 1:2로 같고 그것을 닮음비라고 한다.이렇게 공부를 했었는데 이것 말고도 이러한 것들도 성립할 거란 얘기예요. 예를 들어 작은 삼각형에서 빗변 분의 높이 얼마입니까? 5분의3이죠?큰 삼각형에서 빗변 분의 높이 얼마죠? 10분의6이죠. 이 둘 어때요? 5분의3으로 서로 같다는 거 확인할 수 있고.또 마찬가지로 작은 삼각형에서 예를 들어서 빗변 분의 밑변 해 봅시다. 5분의4죠. 큰 삼각형에서 빗변 분의 밑변 해 보면 얼마죠?10분의8이죠. 근데 이것도 서로 어때? 같아요. 마찬가지로 하나 더 해 볼까요? 작은 삼각형에서 밑변 분의 높이는 4분의3이죠.큰 삼각형에서 밑변 분의 높이는 어때요? 8분의6인데, 이들이 모두 다 어때? 같다는 걸 여러분들이 알 수 있단 얘기예요. 결국 지금 이 둘의 차이는, 같은 거 같아 보이나요? 조금 달라요.작은 삼각형과 큰 삼각형의 두 대응변의 길이의 비를 이야기하는 게 닮음비고, 한 삼각형에서 어떤 두 변을 뽑아서 그 둘의 비랑 큰 삼각형에서 그 둘의 길이의 비는비가 서로 같은 성질이 있다는 게

In [8]:
GOOGLE[1000:3000]

' 수선을 똑같이 보여 주면 바로 2차각성 가평 a b a d는 서로 어떤 닮은 a 닮음이 된다 이건 우리가 중학교 2학년 때 가게가 통이고 그다음에 둘 다 치과를 갖고 있으니까 a 삶이 될 테고 우리가 주의 때문에 어떤 닮음비 일하는 건 뭐냐면 대응하는 변의 길이의 비 왔어요 그래서 그래서 만약에 이게들을 오라고 해 보고 얘기를 11분 삼각형의 닮음 비는 5대 11대 이겠죠 자 그렇다면 우리가이 길을 사라고 나 보면 이게 된 얼마나 됩니까 당연히 그쵸 5일 8일 해야 돼 진짜이 길을 걷고 우리가 6이라고 해보겠습니다 그럼 거꾸로 작은 쪽은 얼마나 됐다 1대 2대 1 얼마 3일 돼야 된다는 걸 여러분들이 할 수 있죠\n 우리는 닮은 비라는 건 작은 삼각형과 큰 삼각형의 대응하는 변의 길이의 비니까 오대식 3대 6\n 4대 8 이것이 모두 다 일대일로 갖고 그것을 닮음비라고 한다 이렇게 공부했었는데 이것 말고도 이런 것들도 성립할 거라 예를 들어 작은 삼각형에서 빗변분의 높이 얼마입니까 큰 삼각형에서 빗변분의 높이 얼마죠 숯불에 유기적 이들 어때요 5분의 3으로 서로 같다라는 확인할 수 있고 또 마찬가지로 자는 삼각형에서 예를 들어서 빗변분의 밑변에 봅시다 5분의 4 적 큰 삼각형에서 변 분에 밑변에 보면 얼마죠 10분에 반이죠 근데 이것도 사러 때 같아요 마찬가지로 하나 더 해 볼까요 작은 삼각형에서 밑변 불에 높이는 4분의 3 이적 큰 삼각형에서 미평 그래도 필요 없대요 8분의 6인데 이들이 모두 다 어때 같다란 걸려 놈들이 알 수 있단 얘기야 결국 지금이 둘의 차이는 같은 거 같아 보이나요\n 달라요 작은 삼각형과 큰 삼각형의 두 대는 변의길이 비를 이야기 한게 닮음비고 1 삼각형에서 어떤 두 병을 뽑아서 그들의 비람 큰 삼각형 둘레 길이 8m 서로 비가 서로 같은 성질이 있다라는게 바로 닮은 직각삼각형에서 성질입니다 이제 우리 가요 내용은 당연히 받아들일 수 있을 테니이 내용에 대해서 간단하게 확인 문자 한번 살펴보고 가도록 하겠습니다 오른쪽 그림과 같은

In [9]:
def decimalnumber_to_korean(m):
    answer = ''
    for _ in m.group(1):
        answer += kopro[int(_)]
    answer += '점'
    for _ in m.group(3):
        answer += kopro[int(_)]
    return answer

In [10]:
def number_to_korean(s):
    #소수점 한국어로
    s = re.sub(r'(\d+)(\.)(\d+)', decimalnumber_to_korean, s)
    
    #정수 한국어로
    fl = re.findall(r'\d+', s)
    fl = set(fl)
    fl = [int(_) for _ in fl]
    fl.sort(reverse=True)
    fl = [str(_) for _ in fl]
    for n in fl:
        s = s.replace(n, kopro[int(n)])
    return s

In [11]:
def mathsymbol_to_korean(s):
    for ms in sd.keys():
        if ms in s:
            s = s.replace(ms, sd[ms])
    return s

In [12]:
def alphabet_to_korean(s):
    for a in ad.keys():
        if a in s:
            s = s.replace(a, ad[a])
    return s

In [13]:
def remove_newlines(s):
    
    return re.sub(r'[\n]+', ' ', s)

In [14]:
def remove_puncuations(s):
    
    return re.sub(r'[\.?,‘’]+', ' ', s)

In [339]:
def insert_dot(text, dot):
    pattern1 = re.compile(r'\S(ㄷㅏ)$')   #다
#     pattern2 = re.compile(r'(.ㅔ|ㅏ|ㅓ|ㅐ|ㅗ|ㅜ)(ㅇㅛ)$') # ㅔ요, ㅏ요, ㅓ요
#     pattern3 = re.compile(r'(ㅆ)(ㅈ|ㅊ)(ㅛ)$') 
#     pattern4 = re.compile(r'(ㅂ|ㅣ)(ㄴㅣㄲㅏ)$')  #ㅂ니까
#     pattern5 = re.compile(r'(ㄴㄷㅔ)$')
    text_list = []
    _1 = 0
    _2 = 0
    _3 = 0
    _4 = 0
    _5 = 0
    _6 = 0
    for _ in text.split(' '):         
        new_ = j2hcj(h2j(_))
        if pattern1.findall(new_):
            text_list.append(_.replace(_, _+dot))
            _1 += 1
#         elif pattern2.findall(new_):
#             text_list.append(_.replace(_, _+dot))
#             _2 += 1
#             list2.append(_)
#         elif pattern3.findall(new_):
#             text_list.append(_.replace(_, _+dot))
#             _3 += 1
#             list3.append(_)
#         elif pattern4.findall(new_):
#             text_list.append(_.replace(_, _+dot))
#             _4 += 1
#         elif pattern5.findall(new_):
#             text_list.append(_.replace(_,_+dot))
#             _5 += 1
        else:
            text_list.append(_)
            _6 += 1
#     print('pattern1 = {}, pattern2= {}, pattern3 = {}, pattern5 = {}, pattern6 = {}'.format(_1,_2,_3,_4,_5,_6)) 
    #print('pattern2 = ',list2)#'\n','list5 = ',list5)
    return text_list    

In [385]:
# EBS 전처리
def EBS_preprocessing(EBS):
    
#     수학기호 한국어로 변환(EBS)
    ebs_math_conversion = mathsymbol_to_korean(EBS)
        
#     숫자 한국어로 변환(EBS)
    ebs_number_conversion = number_to_korean(ebs_math_conversion)
        
#     알파벳 한국어로 변환(EBS)
    ebs_alphabet_conversion = alphabet_to_korean(ebs_number_conversion)
    
#     특수문자 제거
    ebs_punc_removed = remove_puncuations(ebs_alphabet_conversion)
    
#     점 집어넣기
    ebs_dot_inserted = ' '.join(insert_dot(ebs_punc_removed,'.'))

#     띄어쓰기 제거
#     ebs_whitespace_removed = re.sub('[\s]+', '', ebs_dot_inserted)

    return ebs_dot_inserted

In [386]:
# GOOGLE 전처리
def GOOGLE_preprocessing(GOOGLE):
        
#     개행 삭제
    google_newline_removed = remove_newlines(GOOGLE)
    
#     수학기호 한국어로 변환
    google_math_conversion = mathsymbol_to_korean(google_newline_removed)
    
#     숫자 한국어로 변환(GOOGLE)
    google_number_conversion = number_to_korean(google_math_conversion)
        
#     알파벳 한국어로 변환(GOOGLE)
    google_alphabet_conversion = alphabet_to_korean(google_number_conversion)
    
#     특수문자 제거
    google_punc_removed = remove_puncuations(google_alphabet_conversion)
    
#     점 집어넣기
    google_dot_inserted = ' '.join(insert_dot(google_punc_removed,'.'))
    
#     띄어쓰기 제거
#     google_whitespace_removed = re.sub('[\s]+', '', google_dot_inserted)

    return google_dot_inserted

In [387]:
ko_pre_EBS = EBS_preprocessing(EBS) #한글로 치환

In [388]:
ko_pre_GOOGLE = GOOGLE_preprocessing(GOOGLE) #한글로 치환

In [391]:
kkma = Kkma()

In [392]:
ebs_pos = kkma.pos(ko_pre_EBS)

In [393]:
google_pos = kkma.pos(ko_pre_GOOGLE)

In [394]:
def selected_pos(pos_text):
    alist = []
    for _ in pos_text:
        if _[1] in ['NNG','NNP','NNB','NR','NP', 'VV','VA','XR', 'SF']:
            alist.append(_[0])
            continue
    return alist

# 'NNG' : 보통명사,
# 'NNP': 고유명사,
# 'NNB': 일반의존명사,
# 'NR': 수사,
# 'NP': 대명사, 
# 'VV': 동사,
# 'VA': 형용사,
# 'XR': 어근
# 'SF': 마침표

In [395]:
list_ebs_pos = selected_pos(ebs_pos)

In [396]:
list_google_pos = selected_pos(google_pos)

In [399]:
integrated_into_W2V = [list_ebs_pos + list_google_pos]

In [400]:
model = Word2Vec(
    integrated_into_W2V, 
    size = 100,
    min_count=1,
    sg = 1
)
words = list(model.wv.vocab)
print(words)

['반갑', '.', '이비', '에스', '친구', '여러분', '중학', '뉴런', '수학', '삼하', '강의', '되', '손', '석민', '이', '학년', '통하', '만나', '이렇', '앞', '과정', '어떻', '진행', '지', '대하', '오리엔테이션', '확인', '수', '있', '거', '고요', '듣', '오시', '삼', '하', '수업', '것', '같', '경우', '크', '삼개', '대단원', '이루어지', '번째', '삼각비', '단원', '원', '성질', '통계', '일', '마찬가지', '그리하', '도형', '이야기', '다루', '라', '걸', '생각', '오늘', '시간', '삼각', '알', '비', '합', '쳐지', '새', '용어', '나오', '그러', '개념', '공부', '서', '뜻', '시작', '시작하', '터', '중학교', '샘', '끝내', '내용', '가지', '제목', '긷', '닮', '직각', '삼각형', '대응변', '길이', '그다음', '뭐', '면', '갖', '기본적', '가', '자', '앞서', '징검다리', '받아들이', '그렇', '그리', '개', '편하', '제일', '좋', '방법', '무엇', '까', '우리', '한번', '여기', '수선', '긋', '바깥쪽', '하나', '보이', '다르', '빗변', '위', '점', '잡', '직', '에이', '비씨', '에이디', '닮음', '이건', '때', '배우', '공통', '둘', '중이', '닮음비', '대응', '변', '만약', '오라', '보', '십', '오대', '대이', '사', '노', '얼마', '배', '팔', '육', '작', '쪽', '일대', '이대', '삼이', '삼대', '사대', '이것', '다', '이로', '그것', '말', '이러하', '드', '도', '성립', '얘기', '예', '분의', '높이', '오', '얼', '어', '서로', '밑변', '사분', '팔

In [401]:
print(len(words))

4348


In [402]:
model.save("word2vec.model")

In [403]:
model_skip.wv.similar_by_word('콜사인',topn=20)

[('하트', 0.2974466383457184),
 ('강', 0.26863038539886475),
 ('하라', 0.2575220465660095),
 ('거래', 0.2311132252216339),
 ('행사', 0.2310403734445572),
 ('오각형', 0.22995735704898834),
 ('거들', 0.22503654658794403),
 ('명이', 0.2225310206413269),
 ('파리', 0.22091710567474365),
 ('사변형', 0.2181755006313324),
 ('레벨', 0.21610905230045319),
 ('포기', 0.2155865877866745),
 ('비지', 0.21431833505630493),
 ('기린', 0.21308697760105133),
 ('오후', 0.21134428679943085),
 ('진짜', 0.2089933454990387),
 ('나뉘', 0.20857016742229462),
 ('동안', 0.20322656631469727),
 ('박스', 0.19962278008460999),
 ('저희', 0.19928677380084991)]

In [404]:
model_skip.wv.similar_by_word('데이트',topn=20) #대입

[('내면', 0.29617002606391907),
 ('도예', 0.27150610089302063),
 ('여자', 0.2542673945426941),
 ('아이디', 0.250887930393219),
 ('집값', 0.24894005060195923),
 ('약품', 0.24455039203166962),
 ('가도', 0.24440628290176392),
 ('관심', 0.24160043895244598),
 ('철저', 0.22979405522346497),
 ('마', 0.22847720980644226),
 ('비에스', 0.22244691848754883),
 ('보도', 0.21351350843906403),
 ('라니', 0.21278734505176544),
 ('타', 0.21257446706295013),
 ('건물', 0.20092687010765076),
 ('육십이', 0.19910520315170288),
 ('유재', 0.19785979390144348),
 ('일사', 0.19715434312820435),
 ('꺼내', 0.1960379034280777),
 ('걷', 0.19522112607955933)]

In [405]:
model_skip.wv.similar_by_word('직각',topn=20)

[('크', 0.9972876310348511),
 ('삼각형', 0.996359646320343),
 ('작', 0.99507075548172),
 ('이야기', 0.9948509335517883),
 ('얘기', 0.9936521649360657),
 ('때', 0.9916315078735352),
 ('길이', 0.9915724396705627),
 ('비가', 0.991288959980011),
 ('건', 0.9911531805992126),
 ('구하', 0.9909771084785461),
 ('대하', 0.9906612038612366),
 ('누구', 0.990649938583374),
 ('것', 0.990526556968689),
 ('그렇', 0.9901041388511658),
 ('의미', 0.9897381663322449),
 ('이', 0.9896234273910522),
 ('닮음', 0.9895802736282349),
 ('주어지', 0.9894728660583496),
 ('예', 0.9894192218780518),
 ('와이', 0.989398181438446)]

In [406]:
model_skip.wv.similar_by_word('양변',topn=20)

[('치', 0.9995932579040527),
 ('분자', 0.9995322227478027),
 ('해', 0.9995283484458923),
 ('구이', 0.9995207190513611),
 ('가운데', 0.9994694590568542),
 ('성립', 0.9994630217552185),
 ('쓰', 0.9994528293609619),
 ('유리', 0.9994106292724609),
 ('주면', 0.9994081854820251),
 ('팔분의', 0.9994006752967834),
 ('화해', 0.9993913173675537),
 ('같아지', 0.9993869066238403),
 ('정답', 0.9993765950202942),
 ('오가', 0.9993763566017151),
 ('갈', 0.9993747472763062),
 ('고치', 0.9993718266487122),
 ('줄', 0.9993704557418823),
 ('일구', 0.9993643164634705),
 ('나가', 0.9993574619293213),
 ('일대', 0.999346911907196)]

In [407]:
model_skip.wv.similar_by_word('양배추',topn=20)

KeyError: "word '양배추' not in vocabulary"

In [408]:
model_skip.wv.similar_by_word('쓰레기',topn=20) #직'선의기'울기

[('삼과', 0.2644292116165161),
 ('오후', 0.2504929006099701),
 ('잔소리', 0.24822725355625153),
 ('형의', 0.24732816219329834),
 ('관심', 0.2448856234550476),
 ('형', 0.23001109063625336),
 ('묶음', 0.21705400943756104),
 ('게임', 0.21643847227096558),
 ('널', 0.21264441311359406),
 ('요번', 0.2119521200656891),
 ('고난', 0.21170677244663239),
 ('에스라', 0.21082039177417755),
 ('모', 0.2107030302286148),
 ('제가', 0.2101621925830841),
 ('뽑아내', 0.20893943309783936),
 ('비만', 0.20859748125076294),
 ('철저', 0.20766910910606384),
 ('관하', 0.20666716992855072),
 ('기린', 0.20152893662452698),
 ('내기', 0.20131976902484894)]

In [409]:
model_skip.wv.similar_by_word('싸이',topn=20)

[('게이', 0.3162769079208374),
 ('각시', 0.2525727450847626),
 ('이적', 0.23354528844356537),
 ('색칠', 0.22835905849933624),
 ('다하', 0.22682583332061768),
 ('말씀', 0.219784215092659),
 ('이웃', 0.20932167768478394),
 ('사십팔', 0.20162630081176758),
 ('신나', 0.19952860474586487),
 ('신', 0.1966233253479004),
 ('하늘', 0.1964930146932602),
 ('계속', 0.19621184468269348),
 ('살', 0.194468691945076),
 ('회전', 0.19387272000312805),
 ('별표', 0.19344475865364075),
 ('요가', 0.1881333887577057),
 ('샵', 0.18756116926670074),
 ('만이', 0.1833513379096985),
 ('추우', 0.1816176474094391),
 ('채점', 0.17997224628925323)]

In [410]:
model_skip.wv.similar_by_word('임진각',topn=20) #끼인각

[('엑셀', 0.2987455129623413),
 ('헤어지', 0.2625042796134949),
 ('때리', 0.24664302170276642),
 ('맛있', 0.23848284780979156),
 ('책', 0.22064273059368134),
 ('가야', 0.2173275351524353),
 ('오니', 0.21423175930976868),
 ('널', 0.2138901799917221),
 ('올라오', 0.2117902785539627),
 ('엘', 0.20079319179058075),
 ('진짜', 0.20043382048606873),
 ('아이디', 0.19950547814369202),
 ('줍', 0.19844700396060944),
 ('중선', 0.1978684663772583),
 ('요기', 0.1972971260547638),
 ('도영', 0.19683267176151276),
 ('벗', 0.19668056070804596),
 ('막대기', 0.19647076725959778),
 ('피자', 0.18999844789505005),
 ('마을', 0.18769362568855286)]

# 아래는 LSTM wikidocs

In [458]:
ebs_before_split = ' '.join(list_ebs_pos).replace(' .', '.')
ebs_morphs_sentence_list = ebs_before_split.split('.')

google_before_split = ' '.join(list_google_pos).replace(' .', '.')
google_morphs_sentence_list = google_before_split.split('.')

intg_mor_sen_list = ebs_morphs_sentence_list + google_morphs_sentence_list

print(len(intg_mor_sen_list))
intg_mor_sen_list[:20]

5691


['반갑',
 ' 이비 에스 친구 여러분 여러분 중학 뉴런 수학 삼하 강의 되 손 석민',
 ' 이 학년 뉴런 통하 만나 여러분 이렇 만나 되 반갑',
 ' 앞 삼하 과정 어떻 진행 지 대하 여러분 오리엔테이션 강의 통하 확인 수 있 거 고요 오리엔테이션 강의 듣 오시 여러분 삼 학년 과정 하 과정 어떻 수업 확인 수 있 것',
 ' 삼 학년 과정 같 경우 크 삼개 대단원 이루어지 번째 삼각비 단원 번째 원 성질 번째 통계 이루어지',
 ' 일 학년 마찬가지 학년 그리하 도형 대하 이야기 삼하 다루 라 걸 생각 되 고요 오늘 시간 삼각비 삼각 알 비 알 합 쳐지 삼각비 새 용어 나오 그러 개념 공부 앞 서 삼각비 뜻 시작 것 시간',
 ' 오늘 시간 시작하 하 터 중학교 삼 학년 수학 손 샘 끝내 공부 내용 가지 개념 오늘 가지 개념 번째 제목 긷 닮 직각 삼각형 대응변 길이 비 것 있',
 ' 그다음 번째 개념 뭐 면 삼각비 것 삼각비 뜻 번째 개념 앞 공부 내용 직각 삼각형 다루 터 직각 삼각형 갖 기본적 성질 있 성질 알 가 자 삼각비 공부 앞서 징검다리 내용',
 ' 여러분 받아들이 되',
 ' 그렇 닮 직각 삼각형 라 하 닮 직각 삼각형 그리 되 닮 직각 삼각형 개 편하 그리 수 있 제일 좋 방법 무엇 까 생각 우리 이렇 그리 되 거 같 한번 그리',
 ' 여기 수선 긋',
 ' 바깥쪽 직각 삼각형 하나 보이',
 ' 그렇 다르 직각 삼각 빗변 위 점 잡 이렇 수선 긋 직 삼각형 에이 비씨 에이디 이 닮음 에이 에이 닮음 되',
 ' 이건 우리 중학교 학년 때 배우 그렇 에이 공통 그다음 둘 직각 갖 에이 에이 닮음 되 터 우리 중이 때 배우 닮음비 것 뭐 하 대응 변 길이 비 그렇 만약 길이 오라 하 보 길이 십 하 삼각형 닮음비 오대 십 대이 그렇 우리 길이 사 노 길이 얼마 되 그렇 사 배 팔 되 되 길이 우리 육 하',
 ' 작 쪽 얼마 되 일대 이대 일 얼마 삼이 되 되 것 여러분 알 수 있 우리 닮음비 것 작 삼각형 크 삼각형 대응 변 길이

In [429]:
import os
from collections import Counter
from time import time

import numpy as np
import pandas as pd
from keras.layers import Dense, Dot, Embedding, Input, Reshape
from keras.models import Model
from keras.preprocessing.sequence import skipgrams
from nltk.corpus import stopwords

np.random.seed(777)
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'

Using TensorFlow backend.


# Text Generation using RNN

### https://wikidocs.net/45101

In [481]:
text = intg_mor_sen_list
text[:100]

['반갑',
 ' 이비 에스 친구 여러분 여러분 중학 뉴런 수학 삼하 강의 되 손 석민',
 ' 이 학년 뉴런 통하 만나 여러분 이렇 만나 되 반갑',
 ' 앞 삼하 과정 어떻 진행 지 대하 여러분 오리엔테이션 강의 통하 확인 수 있 거 고요 오리엔테이션 강의 듣 오시 여러분 삼 학년 과정 하 과정 어떻 수업 확인 수 있 것',
 ' 삼 학년 과정 같 경우 크 삼개 대단원 이루어지 번째 삼각비 단원 번째 원 성질 번째 통계 이루어지',
 ' 일 학년 마찬가지 학년 그리하 도형 대하 이야기 삼하 다루 라 걸 생각 되 고요 오늘 시간 삼각비 삼각 알 비 알 합 쳐지 삼각비 새 용어 나오 그러 개념 공부 앞 서 삼각비 뜻 시작 것 시간',
 ' 오늘 시간 시작하 하 터 중학교 삼 학년 수학 손 샘 끝내 공부 내용 가지 개념 오늘 가지 개념 번째 제목 긷 닮 직각 삼각형 대응변 길이 비 것 있',
 ' 그다음 번째 개념 뭐 면 삼각비 것 삼각비 뜻 번째 개념 앞 공부 내용 직각 삼각형 다루 터 직각 삼각형 갖 기본적 성질 있 성질 알 가 자 삼각비 공부 앞서 징검다리 내용',
 ' 여러분 받아들이 되',
 ' 그렇 닮 직각 삼각형 라 하 닮 직각 삼각형 그리 되 닮 직각 삼각형 개 편하 그리 수 있 제일 좋 방법 무엇 까 생각 우리 이렇 그리 되 거 같 한번 그리',
 ' 여기 수선 긋',
 ' 바깥쪽 직각 삼각형 하나 보이',
 ' 그렇 다르 직각 삼각 빗변 위 점 잡 이렇 수선 긋 직 삼각형 에이 비씨 에이디 이 닮음 에이 에이 닮음 되',
 ' 이건 우리 중학교 학년 때 배우 그렇 에이 공통 그다음 둘 직각 갖 에이 에이 닮음 되 터 우리 중이 때 배우 닮음비 것 뭐 하 대응 변 길이 비 그렇 만약 길이 오라 하 보 길이 십 하 삼각형 닮음비 오대 십 대이 그렇 우리 길이 사 노 길이 얼마 되 그렇 사 배 팔 되 되 길이 우리 육 하',
 ' 작 쪽 얼마 되 일대 이대 일 얼마 삼이 되 되 것 여러분 알 수 있 우리 닮음비 것 작 삼각형 크 삼각형 대응 변 길이

In [482]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 4348


In [493]:
sequences = list()

for line in text: # 1,214 개의 샘플에 대해서 샘플을 1개씩 가져온다.
    encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:20] # 11개의 샘플 출력

[[233, 259],
 [233, 259, 504],
 [233, 259, 504, 20],
 [233, 259, 504, 20, 20],
 [233, 259, 504, 20, 20, 1283],
 [233, 259, 504, 20, 20, 1283, 563],
 [233, 259, 504, 20, 20, 1283, 563, 198],
 [233, 259, 504, 20, 20, 1283, 563, 198, 909],
 [233, 259, 504, 20, 20, 1283, 563, 198, 909, 272],
 [233, 259, 504, 20, 20, 1283, 563, 198, 909, 272, 1],
 [233, 259, 504, 20, 20, 1283, 563, 198, 909, 272, 1, 910],
 [233, 259, 504, 20, 20, 1283, 563, 198, 909, 272, 1, 910, 2074],
 [50, 403],
 [50, 403, 563],
 [50, 403, 563, 217],
 [50, 403, 563, 217, 310],
 [50, 403, 563, 217, 310, 20],
 [50, 403, 563, 217, 310, 20, 31],
 [50, 403, 563, 217, 310, 20, 31, 310],
 [50, 403, 563, 217, 310, 20, 31, 310, 1]]

In [ ]:
print(t.word_index[:100])

In [494]:
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : 물


In [495]:
max_len=max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 436


In [496]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[  0   0   0 ...   0 233 259]
 [  0   0   0 ... 233 259 504]
 [  0   0   0 ... 259 504  20]]


In [497]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [498]:
print(X[:3])

[[  0   0   0 ...   0   0 233]
 [  0   0   0 ...   0 233 259]
 [  0   0   0 ... 233 259 504]]


In [499]:
print(y[:3]) # 레이블

[259 504  20]


In [500]:
vocab_size

4348

In [501]:
y = to_categorical(y, num_classes=vocab_size)

In [502]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_len-1))
# y데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(LSTM(10))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=5, verbose=2)

Epoch 1/5


In [ ]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [ ]:
sentence_generation(model, t, '이 병 분에 높이', 5)